In [215]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import numpy as np

warnings.simplefilter(action='ignore', category=FutureWarning) # ignores warnings from pd.append()

In [216]:
# column names
col_names = ['Titel', 'Bevilliget beløb', 'Modtager', 'Institution', 'Virkemidler', 'Område', 'År', 'Beskrivelse']

# create an Empty DataFrame object
df = pd.DataFrame(columns = col_names)

In [217]:
URL = "https://dff.dk/forskningsprojekter?b_start:int="

for i in range(0, 4390, 10):

    print("Scraping page: "+ str(int(float(i/10)))) # prints page being scraped at the moment

    page = requests.get(URL+str(i))
    soup = BeautifulSoup(page.text, "html.parser")

    # Load page as html
    bevillinger = soup.find_all("div", class_ = "result-item")

    # Retrieve all grants from first page
    for bevilling in bevillinger:
        titel = bevilling.find("h2", class_ = "result-title").text.strip()
        beløb = bevilling.find("div", class_ = "col-sm-2 text-right result-amount").text.replace("Bevilget beløb", "").strip()
        
        modtager_liste = bevilling.find("div", class_ = "col-xs-6 col-sm-12").text.strip().split(sep="\n\n") # remove whitespaces and split based on "\n\n"
        modtager_liste = [s.strip() for s in modtager_liste] # strip whitespace for both Modtager and Institution

        try:
            modtager = modtager_liste[0]
            institution = modtager_liste[1]
        except IndexError:
            modtager = modtager_liste[0]
            institution = ""
        
        info_liste = bevilling.find("ul", class_ = "listing-horizontal").text.strip().split(sep="\n\n") # remove whitespaces and split based on "\n\n"
        info_liste = [s.strip() for s in info_liste] # strip whitespace for both Virkemiddel, Område and År

        virkemidler = info_liste[0]
        område = info_liste[1]
        periode = info_liste[2]

        try:
            beskrivelse = bevilling.find("div", class_ = "row result-body").text.strip()
        except AttributeError:
            beskrivelse = ""

        # append rows to an empty DataFrame
        df = df.append({'Titel'             : titel,
                        'Bevilliget beløb'  : beløb,
                        'Modtager'          : modtager,
                        'Institution'       : institution,
                        'Virkemidler'       : virkemidler,
                        'Område'            : område,
                        'År'                : periode,
                        'Beskrivelse'       : beskrivelse},
                        ignore_index = True)

Scraping page: 0
Scraping page: 1
Scraping page: 2
Scraping page: 3
Scraping page: 4
Scraping page: 5
Scraping page: 6
Scraping page: 7
Scraping page: 8
Scraping page: 9
Scraping page: 10
Scraping page: 11
Scraping page: 12
Scraping page: 13
Scraping page: 14
Scraping page: 15
Scraping page: 16
Scraping page: 17
Scraping page: 18
Scraping page: 19
Scraping page: 20
Scraping page: 21
Scraping page: 22
Scraping page: 23
Scraping page: 24
Scraping page: 25
Scraping page: 26
Scraping page: 27
Scraping page: 28
Scraping page: 29
Scraping page: 30
Scraping page: 31
Scraping page: 32
Scraping page: 33
Scraping page: 34
Scraping page: 35
Scraping page: 36
Scraping page: 37
Scraping page: 38
Scraping page: 39
Scraping page: 40
Scraping page: 41
Scraping page: 42
Scraping page: 43
Scraping page: 44
Scraping page: 45
Scraping page: 46
Scraping page: 47
Scraping page: 48
Scraping page: 49
Scraping page: 50
Scraping page: 51
Scraping page: 52
Scraping page: 53
Scraping page: 54
Scraping page: 55
Sc

In [218]:
# convert column År to numeric
df["År"] = pd.to_numeric(df["År"])

# convert column Bevilliget beløb to numeric
df["Bevilliget beløb"] = df["Bevilliget beløb"].str.replace("kr", "").str.strip()
df["Bevilliget beløb"] = df["Bevilliget beløb"].str.replace(".", "")
df["Bevilliget beløb"] = pd.to_numeric(df["Bevilliget beløb"])

# convert first character of each word to uppercase in column Institution
df["Institution"] = df["Institution"].str.title()

# data wrangling
df.loc[df['Institution'].isin(['Købehavns Universitet', 'Københans Universitet'])] = "Københavns Universitet" # replaces values
df.loc[df['Institution'] == "Danmarks Teknisk Universitet"] = "Danmarks Tekniske Universitet" # # replaces values

# trim extra space between substrings
df["Institution"] = df["Institution"].str.replace(' +', ' ')

In [224]:
# sense check
df.tail()

,Titel,Bevilliget beløb,Modtager,Institution,Virkemidler,Område,År,Beskrivelse
4145,Enabling Ultra Deep Hydrodesulphurization by N...,10781874,Ib Chorkendorff,Danmarks Tekniske Universitet,Øvrige forskningsprojekter,Teknologi og Produktion,2013,Alle olieprodukter renses i dag for svovl for ...
4146,Acute stroke research,717359,Hanne Krarup Christensen,"Bispebjerg Hospital, Neurologisk Afdeling",Delestillinger,Sundhed og Sygdom,2013,Aktuelle ansøgning angår frikøb af overlæge Ha...
4147,Atherosclerotic cardiovascular disease in HIV-...,764683,Anne-Mette Lebech,"Hvidovre Hospital, Infektionsmedicinsk Afdeling",Delestillinger,Sundhed og Sygdom,2013,Behandling af HIV positive patienter med anti-...
4148,Epigenetic modulation of mechanisms involved i...,829294,Ole Schmeltz Søgaard,"Aarhus Universitetshospital, Infektionsmedicin...",Delestillinger,Sundhed og Sygdom,2013,HIV infektion behandles i dag med en kombinati...
4149,Novel mechanisms of insulin resistance and mit...,665923,Kurt Højlund,"Odense Universitetshospital, Endokrinologisk A...",Delestillinger,Sundhed og Sygdom,2013,Insulinresistens (IR) i muskelvæv spiller en v...


In [225]:
# create dictionary for Region
regioner_dict = {
    "Region Nordjylland": [ "Aalborg", "Nordjylland", "Nordjyllands", "Ålborg"],

    "Region Midtjylland": [ "Aarhus", "Midtjylland", "Midtjyllands", "Via University College", "Danmarks Industrimuseum",
                            "Horsens", "Århus", "Central Region Denmark", "KORA", "Moesgård Museum",
                            "Center for Rusmiddelforskning"],

    "Region Syddanmark" : [ "Syddansk", "Southern Denmark", "Odense", "Kolding", "Syddanmark", "Syddanmarks",
                            "Dansk Sprognævn", "Sdu"],

    "Region Sjælland"   : [ "Sjælland", "Roskilde", "Slagelse", "Professionshøjskolen Absalon", "Dansk Historisk Fællesråd",
                            "Sjællands", "Lolland", "Falster", "Næstved", "Holbæk", "Nykøbing"],

    "Region Hovedstaden": [ "København", "Copenhagen", "Ku", "Københavns", "Nordsjælland", "Nordsjællands",
                            "Danmarks Tekniske Universitet", "Dtu", "Technical University Of Denmark",
                            "Hovedstaden", "Hovedstadens","Rigshospitalet", "Bispebjerg",
                            "Herlev", "Gentofte", "Hvidovre", "Glostrup", "Frederiksberg", "Hillerød",
                            "Forsvarsakademiet", "Statens Serum Institut", "Ssi",
                            "Det Kongelige Danske", "Kadk", "Vive", "Sfi", "Statens Museum For Kunst", "Smk",
                            "University College Capital", "Ucc", "Center For Kræftforskning",
                            "Diis", "Geus", "Dignity", "Selskabet for Skole- Og Uddannelseshistorie",
                            "Statens Naturhistoriske Museum", "Professionshøjskolen Metropol", "Kræftens Bekæmpelse",
                            "Dansk Institut Mod Tortur", "Rigsarkivet", "Det Kongelige Akademi", "Psykiatrisk Center Sct. Hans",
                            "De Nationale Geologiske Undersøgelser For Danmark Og Grønland", "Nationalmuseet",
                            "Dansk Institut For Internationale Studier", "Den Selvejende Institution Dansk Børneastma Center",
                            "Institut For Menneskerettigheder", "Det Kongelige Bibliotek", "Danish archives, museums and libraries",
                            "Det Nationale Forskningscenter For Arbejdsmiljø", "Forskningscenter For Forebyggelse Og Sundhed",
                            "Rockwool Fonden", "Selskabet Til Forskning I Arbejderbevægelsens Historie",
                            "Universitets-Jubilæets Danske Samfund", "Dhi", "Statens Arkiver",
                            "Det Nationale Institut For Kommuners Og Regioners Analyse Og Forskning"]
}

In [226]:
# create empty column for Region
df["Region"] = np.nan

# compares dict values to Institution string and append with key
for i in range(len(df["Institution"])): # loops over instution
    
    for substring_list in regioner_dict.values(): # loops over list of values in dictionary
        
        if any(substring in df["Institution"][i] for substring in substring_list) == True: # if institution matches any of the substrings then find the key
            region = list(regioner_dict.keys())[list(regioner_dict.values()).index(substring_list)] #
            df["Region"][i] = region

/var/folders/d7/thxq_x3543scnpsc7pyl2fs40000gn/T/ipykernel_4261/1367416360.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Region"][i] = region


In [230]:
df.tail()

,Titel,Bevilliget beløb,Modtager,Institution,Virkemidler,Område,År,Beskrivelse,Region
4145,Enabling Ultra Deep Hydrodesulphurization by N...,10781874,Ib Chorkendorff,Danmarks Tekniske Universitet,Øvrige forskningsprojekter,Teknologi og Produktion,2013,Alle olieprodukter renses i dag for svovl for ...,Region Hovedstaden
4146,Acute stroke research,717359,Hanne Krarup Christensen,"Bispebjerg Hospital, Neurologisk Afdeling",Delestillinger,Sundhed og Sygdom,2013,Aktuelle ansøgning angår frikøb af overlæge Ha...,Region Hovedstaden
4147,Atherosclerotic cardiovascular disease in HIV-...,764683,Anne-Mette Lebech,"Hvidovre Hospital, Infektionsmedicinsk Afdeling",Delestillinger,Sundhed og Sygdom,2013,Behandling af HIV positive patienter med anti-...,Region Hovedstaden
4148,Epigenetic modulation of mechanisms involved i...,829294,Ole Schmeltz Søgaard,"Aarhus Universitetshospital, Infektionsmedicin...",Delestillinger,Sundhed og Sygdom,2013,HIV infektion behandles i dag med en kombinati...,Region Midtjylland
4149,Novel mechanisms of insulin resistance and mit...,665923,Kurt Højlund,"Odense Universitetshospital, Endokrinologisk A...",Delestillinger,Sundhed og Sygdom,2013,Insulinresistens (IR) i muskelvæv spiller en v...,Region Syddanmark


In [231]:
# save to csv
df.to_csv('dff.csv', sep=',', header=True, index=False)

# import csv
df_new = pd.read_csv('dff.csv')
df_new.tail()

,Titel,Bevilliget beløb,Modtager,Institution,Virkemidler,Område,År,Beskrivelse,Region
4145,Enabling Ultra Deep Hydrodesulphurization by N...,10781874,Ib Chorkendorff,Danmarks Tekniske Universitet,Øvrige forskningsprojekter,Teknologi og Produktion,2013,Alle olieprodukter renses i dag for svovl for ...,Region Hovedstaden
4146,Acute stroke research,717359,Hanne Krarup Christensen,"Bispebjerg Hospital, Neurologisk Afdeling",Delestillinger,Sundhed og Sygdom,2013,Aktuelle ansøgning angår frikøb af overlæge Ha...,Region Hovedstaden
4147,Atherosclerotic cardiovascular disease in HIV-...,764683,Anne-Mette Lebech,"Hvidovre Hospital, Infektionsmedicinsk Afdeling",Delestillinger,Sundhed og Sygdom,2013,Behandling af HIV positive patienter med anti-...,Region Hovedstaden
4148,Epigenetic modulation of mechanisms involved i...,829294,Ole Schmeltz Søgaard,"Aarhus Universitetshospital, Infektionsmedicin...",Delestillinger,Sundhed og Sygdom,2013,HIV infektion behandles i dag med en kombinati...,Region Midtjylland
4149,Novel mechanisms of insulin resistance and mit...,665923,Kurt Højlund,"Odense Universitetshospital, Endokrinologisk A...",Delestillinger,Sundhed og Sygdom,2013,Insulinresistens (IR) i muskelvæv spiller en v...,Region Syddanmark
